# 🚗 Dynamic Pricing for Urban Parking Lots
## Capstone Project – Summer Analytics 2025
### Niladri Banerjee

### 🔹 Step 1: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
output_notebook()

### 🔹 Step 2: Load Dataset

In [ ]:
# Assuming running in Google Colab
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv('Modified - modified.csv')
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'])
df = df.sort_values('Timestamp')
df.head()

### 🔹 Step 3: Model 1 – Linear Pricing Based on Occupancy

In [ ]:
BASE_PRICE = 10
ALPHA = 2
df['LinearPrice'] = BASE_PRICE + ALPHA * (df['Occupancy'] / df['Capacity'])
df['LinearPrice'] = df['LinearPrice'].round(2)
df[['Timestamp', 'Occupancy', 'LinearPrice']].head()

### 🔹 Step 4: Model 2 – Demand-Based Dynamic Pricing

In [ ]:
def traffic_to_score(traffic):
    if traffic == 'low': return 0.2
    elif traffic == 'average': return 0.5
    else: return 1.0

def vehicle_to_weight(vehicle):
    return {'car': 1.0, 'bike': 0.7, 'truck': 1.5}.get(vehicle, 1.0)

df['TrafficScore'] = df['TrafficConditionNearby'].apply(traffic_to_score)
df['VehicleWeight'] = df['VehicleType'].apply(vehicle_to_weight)

# Demand calculation
df['Demand'] = (0.4 * (df['Occupancy'] / df['Capacity']) +
                0.2 * df['QueueLength'] -
                0.2 * df['TrafficScore'] +
                0.1 * df['IsSpecialDay'] +
                0.1 * df['VehicleWeight'])

# Normalize Demand
d_min, d_max = df['Demand'].min(), df['Demand'].max()
df['NormDemand'] = (df['Demand'] - d_min) / (d_max - d_min)

# Price = Base * (1 + lambda * NormDemand)
LAMBDA = 1
df['DemandPrice'] = BASE_PRICE * (1 + LAMBDA * df['NormDemand'])
df['DemandPrice'] = df['DemandPrice'].clip(lower=5, upper=20).round(2)
df[['Timestamp', 'Demand', 'NormDemand', 'DemandPrice']].head()

### 🔹 Step 5: Visualize with Bokeh

In [ ]:
p = figure(x_axis_type='datetime', width=800, height=350, title='Price Comparison Over Time')
p.line(df['Timestamp'], df['LinearPrice'], legend_label='Linear Price', color='blue')
p.line(df['Timestamp'], df['DemandPrice'], legend_label='Demand Price', color='green')
p.legend.location = 'top_left'
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Price (USD)'
show(p)

### 🔹 Step 6: Conclusion
- Dynamic pricing helps balance demand and maximize usage.
- Model 2 provides smoother and context-aware pricing.
- Model 3 can be extended with competitor proximity using lat-long.